In [4]:
default_installations = True
if default_installations:
    !pip -q install langchain langchain_community
    !pip -q install sentence_transformers
    !pip -q install faiss-cpu
    !pip -q install openai==0.28.1
else:
    import requests

    text_file_path = "requirements__Ch9_Advanced_LangChain_Configurations_and_Pipeline.txt"
    url = "https://raw.githubusercontent.com/PacktPublishing/Mastering-NLP-from-Foundations-to-LLMs/main/Chapter9_notebooks/" + text_file_path
    res = requests.get(url)
    with open(text_file_path, "w") as f:
        f.write(res.text)
    !pip install -r requirements__Ch9_Advanced_LangChain_Configurations_and_Pipeline.txt

In [5]:
import requests
import textwrap
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

In [6]:
text_file_path = "mocked_up_physician_records_ko.csv"
url = "https://raw.githubusercontent.com/corazzon/Mastering-NLP-from-Foundations-to-LLMs/refs/heads/main/Chapter8_notebooks/" + text_file_path
res = requests.get(url)
with open(text_file_path, "w") as f:
    f.write(res.text)

In [7]:
text_loader = TextLoader(text_file_path)
documents = text_loader.load()

In [8]:
print(type(documents[0]))

<class 'langchain_core.documents.base.Document'>


In [9]:
len(documents)

1

In [10]:
print(documents[0].page_content[0:2000])

"Title: Mocked up record
담당의: ABC 의사
날짜: 2099년 6월 25일
환자 ID: 987654321
주 증상: 복통
현재 질환 이력:
환자 John Anderson 씨(42세, 남성)는 복통을 주 증상으로 내원하였습니다. 기혼자로 아내와 두 자녀와 함께 살고 있습니다. Anderson 씨는 약 2주 전 유럽 출장에서 돌아왔으며, 여행 중 호흡기 증상이나 아픈 사람과의 접촉은 없었다고 합니다.
진료 중 Anderson 씨는 아버지가 60대에 심근경색을 앓은 심혈관 질환 가족력이 있으며, 외할머니는 제2형 당뇨병을 앓았다고 밝혔습니다. 본인은 만성 질환, 수술 경험이나 입원 이력은 없다고 합니다.
주 증상에 대해 Anderson 씨는 우하복부에 둔하고 간헐적인 통증이 있으며, 통증 강도는 10점 만점에 5점 정도라고 설명합니다. 신체 활동 시 통증이 악화되고 식사 후에도 더 심해지는 경향이 있습니다. 구역질, 구토, 배변 습관의 변화와 같은 관련 증상은 없다고 합니다.
제공된 정보를 바탕으로 Anderson 씨의 복통 원인을 파악하기 위한 추가 평가와 진단 검사가 진행될 예정입니다."
"Title: Mocked up record
담당의: ABC 의사
날짜: 2099년 11월 15일
환자 ID: 123456789
주 증상: 피로감과 관절통
현재 질환 이력:
환자 Sarah Thompson 씨(57세, 여성)는 피로감과 관절통을 호소하며 내원하였습니다. 홀로 거주 중인 미망인으로, 최근 해외여행 이력은 없습니다.
진료 중 Thompson 씨는 자매가 류마티스 관절염 진단을 받은 자가면역 질환 가족력이 있다고 밝혔습니다. 또한 본인은 갑상선 기능 저하증이 있어 갑상선 호르몬 대체 요법으로 관리 중이라고 합니다.
주 증상에 대해 Thompson 씨는 일상 활동에 지장을 줄 정도로 지속적이고 심한 피로감을 느끼며, 그 정도는 10점 만점에 8점이라고 표현했습니다. 또한 주로 무릎과 손목에 관절통이 있는데, 아침에 더 심하고 하루 중 활동하면서 점

In [11]:
split_text_by = '"Title: Mocked up record'
chunk_size = 700
chunk_overlap = 0

In [12]:
text_splitter = CharacterTextSplitter(chunk_size=chunk_size,
                                      chunk_overlap=chunk_overlap,
                                      separator=split_text_by)
splitted_docs = text_splitter.split_documents(documents)

In [13]:
len(splitted_docs)

4

In [14]:
print(splitted_docs[0].page_content)

담당의: ABC 의사
날짜: 2099년 6월 25일
환자 ID: 987654321
주 증상: 복통
현재 질환 이력:
환자 John Anderson 씨(42세, 남성)는 복통을 주 증상으로 내원하였습니다. 기혼자로 아내와 두 자녀와 함께 살고 있습니다. Anderson 씨는 약 2주 전 유럽 출장에서 돌아왔으며, 여행 중 호흡기 증상이나 아픈 사람과의 접촉은 없었다고 합니다.
진료 중 Anderson 씨는 아버지가 60대에 심근경색을 앓은 심혈관 질환 가족력이 있으며, 외할머니는 제2형 당뇨병을 앓았다고 밝혔습니다. 본인은 만성 질환, 수술 경험이나 입원 이력은 없다고 합니다.
주 증상에 대해 Anderson 씨는 우하복부에 둔하고 간헐적인 통증이 있으며, 통증 강도는 10점 만점에 5점 정도라고 설명합니다. 신체 활동 시 통증이 악화되고 식사 후에도 더 심해지는 경향이 있습니다. 구역질, 구토, 배변 습관의 변화와 같은 관련 증상은 없다고 합니다.
제공된 정보를 바탕으로 Anderson 씨의 복통 원인을 파악하기 위한 추가 평가와 진단 검사가 진행될 예정입니다."


In [15]:
embeddings = HuggingFaceEmbeddings(model_name="upskyy/e5-small-korean")

/var/folders/3g/9z2y6vy916ncn7tzc7rkg8140000gn/T/ipykernel_62255/2092518420.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="upskyy/e5-small-korean")


In [16]:
vector_db = FAISS.from_documents(splitted_docs, embeddings)

In [17]:
query1 = "8월에 출산 예정인 임산부 환자가 있나요?"
docs = vector_db.similarity_search(query1)
print(textwrap.fill(str(docs[0].page_content), width=100, replace_whitespace=False))

담당의: ABC 의사
날짜: 2099년 7월 10일
환자 ID: 246813579
주 증상: 임신 정기 검진
현재 질환 이력:
환자 Emily Adams 씨(30세, 여성)는 정기
임신 검진을 위해 내원하였습니다. 현재 임신 32주차로, 2099년 8월 27일이 출산 예정일입니다. 기혼자로 남편과 함께 살고 있습니다.
진료 중 Adams 씨는 어머니가 임신
중 임신성 당뇨를 앓았던 가족력이 있다고 밝혔습니다. 본인은 중요한 병력이나 수술 경험, 이전 임신에서의 합병증은 없다고 합니다.
주 증상에 대해 Adams 씨는 임신 3기에 흔히
나타나는 요통, 잦은 소변, 가끔의 속쓰림 등의 불편함을 호소합니다. 질 출혈, 심한 복통, 태아 움직임의 특이한 변화는 없다고 합니다. 임신 중 전반적인 건강 관리를 위해 균형
잡힌 식단과 규칙적인 운동을 유지하고 있다고 합니다.
제공된 정보를 바탕으로 Adams 씨의 임신 경과를 확인하기 위한 정기 산전 검사가 진행될 예정입니다. 혈압, 체중 증가,
자궁저 높이 측정, 태아 심장 박동 모니터링 등을 평가할 것입니다. 건강한 출산과 순조로운 육아를 위해 출산 준비, 모유 수유, 산후 관리에 대한 상담도 함께 이루어질
예정입니다."


In [18]:
query2 = "출산 예정일이 9월인 임산부가 있나요?"
docs = vector_db.similarity_search(query2)
print(textwrap.fill(str(docs[0].page_content), width=100, replace_whitespace=False))

담당의: ABC 의사
날짜: 2099년 7월 10일
환자 ID: 246813579
주 증상: 임신 정기 검진
현재 질환 이력:
환자 Emily Adams 씨(30세, 여성)는 정기
임신 검진을 위해 내원하였습니다. 현재 임신 32주차로, 2099년 8월 27일이 출산 예정일입니다. 기혼자로 남편과 함께 살고 있습니다.
진료 중 Adams 씨는 어머니가 임신
중 임신성 당뇨를 앓았던 가족력이 있다고 밝혔습니다. 본인은 중요한 병력이나 수술 경험, 이전 임신에서의 합병증은 없다고 합니다.
주 증상에 대해 Adams 씨는 임신 3기에 흔히
나타나는 요통, 잦은 소변, 가끔의 속쓰림 등의 불편함을 호소합니다. 질 출혈, 심한 복통, 태아 움직임의 특이한 변화는 없다고 합니다. 임신 중 전반적인 건강 관리를 위해 균형
잡힌 식단과 규칙적인 운동을 유지하고 있다고 합니다.
제공된 정보를 바탕으로 Adams 씨의 임신 경과를 확인하기 위한 정기 산전 검사가 진행될 예정입니다. 혈압, 체중 증가,
자궁저 높이 측정, 태아 심장 박동 모니터링 등을 평가할 것입니다. 건강한 출산과 순조로운 육아를 위해 출산 준비, 모유 수유, 산후 관리에 대한 상담도 함께 이루어질
예정입니다."


In [19]:
query3 = "최근에 여행을 다녀온 환자는 누구인가요?"
docs = vector_db.similarity_search(query3)
print(textwrap.fill(str(docs[0].page_content), width=100, replace_whitespace=False))

담당의: ABC 의사
날짜: 2099년 11월 15일
환자 ID: 123456789
주 증상: 피로감과 관절통
현재 질환 이력:
환자 Sarah Thompson 씨(57세,
여성)는 피로감과 관절통을 호소하며 내원하였습니다. 홀로 거주 중인 미망인으로, 최근 해외여행 이력은 없습니다.
진료 중 Thompson 씨는 자매가 류마티스 관절염 진단을 받은
자가면역 질환 가족력이 있다고 밝혔습니다. 또한 본인은 갑상선 기능 저하증이 있어 갑상선 호르몬 대체 요법으로 관리 중이라고 합니다.
주 증상에 대해 Thompson 씨는 일상
활동에 지장을 줄 정도로 지속적이고 심한 피로감을 느끼며, 그 정도는 10점 만점에 8점이라고 표현했습니다. 또한 주로 무릎과 손목에 관절통이 있는데, 아침에 더 심하고 하루 중
활동하면서 점차 나아진다고 합니다. 관절 부위의 붓기나 붉어짐은 없다고 합니다.
임상 소견을 바탕으로 Thompson 씨의 피로감과 관절통의 원인을 찾기 위한 추가 검사가
필요합니다. 혈액 검사, 영상 검사 및 필요시 전문의 상담을 진행할 예정입니다."


In [20]:
query4 = "검사실 검사가 필요한 환자는 누구인가요?"
docs = vector_db.similarity_search(query4)
print(textwrap.fill(str(docs[0].page_content), width=100, replace_whitespace=False))

담당의: ABC 의사
날짜: 2099년 11월 15일
환자 ID: 123456789
주 증상: 피로감과 관절통
현재 질환 이력:
환자 Sarah Thompson 씨(57세,
여성)는 피로감과 관절통을 호소하며 내원하였습니다. 홀로 거주 중인 미망인으로, 최근 해외여행 이력은 없습니다.
진료 중 Thompson 씨는 자매가 류마티스 관절염 진단을 받은
자가면역 질환 가족력이 있다고 밝혔습니다. 또한 본인은 갑상선 기능 저하증이 있어 갑상선 호르몬 대체 요법으로 관리 중이라고 합니다.
주 증상에 대해 Thompson 씨는 일상
활동에 지장을 줄 정도로 지속적이고 심한 피로감을 느끼며, 그 정도는 10점 만점에 8점이라고 표현했습니다. 또한 주로 무릎과 손목에 관절통이 있는데, 아침에 더 심하고 하루 중
활동하면서 점차 나아진다고 합니다. 관절 부위의 붓기나 붉어짐은 없다고 합니다.
임상 소견을 바탕으로 Thompson 씨의 피로감과 관절통의 원인을 찾기 위한 추가 검사가
필요합니다. 혈액 검사, 영상 검사 및 필요시 전문의 상담을 진행할 예정입니다."


In [21]:
import sys

local_vars = list(locals().items())
for var, obj in local_vars:
    if (sys.getsizeof(obj)) > 999:
        print(var, sys.getsizeof(obj))

TextLoader 1704
CharacterTextSplitter 1704
HuggingFaceEmbeddings 1704
FAISS 1704


In [22]:
import gc

del CharacterTextSplitter
del HuggingFaceEmbeddings
del TextLoader
del FAISS
gc.collect()

169

In [23]:
openai_api_key = "..."

In [24]:
!pip -q install openai gpt4all==1.0.12

In [25]:
import os
import langchain
from langchain.chains.question_answering import load_qa_chain
from langchain import HuggingFaceHub
from langchain.chat_models import ChatOpenAI
from langchain.llms import GPT4All

ModuleNotFoundError: No module named 'langchain.chains'

In [ ]:
paid_vs_free = "paid"
path_to_bin = "./nous-hermes-13b.ggmlv3.q4_0.bin"
backend_llm = "llama"

In [ ]:
if paid_vs_free == "paid":
    os.environ["OPENAI_API_KEY"] = openai_api_key
    llm = ChatOpenAI()
elif paid_vs_free == "free":
    llm = GPT4All(
        model=path_to_bin,
        max_tokens=1000,
        # backend=backend_llm,
        verbose=False)

In [ ]:
chain = load_qa_chain(llm, chain_type="stuff")

In [ ]:
import langchain

langchain.debug = True

In [ ]:
current_query = query1
print(current_query)
docs = vector_db.similarity_search(current_query, k=4)
response = chain.invoke({"input_documents": docs, "question": current_query})
response

In [ ]:
docs[0]

In [ ]:
current_query = query2
print(current_query)
docs = vector_db.similarity_search(current_query, k=3)
response = chain.invoke({"input_documents": docs, "question": current_query})
response

In [ ]:
current_query = query3
print(current_query)
docs = vector_db.similarity_search(current_query)
response = chain.invoke({"input_documents": docs, "question": current_query})
response

In [ ]:
docs[1]

In [ ]:
current_query = query4
print(current_query)
docs = vector_db.similarity_search(current_query)
response = chain.invoke({"input_documents": docs, "question": current_query})
response

In [ ]:
docs[0]

In [ ]:
current_query = "확실하게 검사실 검사를 필요로 하는 환자는 누구인가요?"
print(current_query)
docs = vector_db.similarity_search(current_query)
response = chain.invoke({"input_documents": docs, "question": current_query})
response